In [18]:
import uuid
import pprint
import logging
import boto3
import json
import time

In [22]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [33]:
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId='CDL0KSVUWE',
    agentAliasId='SY5ZUHIIQ2', 
    sessionId=session_id,
    enableTrace=False, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(f"Trace event: {event['trace']}")
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-08-15 08:40:10,944] p35999 {1893165513.py:15} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 15 Aug 2025 15:40:10 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '20d1be7c-79ee-11f0-9b7f-02b9d5960dc8',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '7ca914a2-768a-4dbe-af6c-4d023e880544'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7ca914a2-768a-4dbe-af6c-4d023e880544',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x10d394c50>,
 'contentType': 'application/json',
 'sessionId': '20d1be7c-79ee-11f0-9b7f-02b9d5960dc8'}


[2025-08-15 08:40:13,732] p35999 {1893165513.py:22} INFO - Final answer ->
The employee with employee_id 1 has 10 vacation days available.
